# Model Free Reinforcement Learning

----

Model free Reinforcement Learning operates policy evaluation, policy imporvement and policy iteration without given model(Reward function and/or transition function).

# Bellman Equations (model based)

----

- Bellman equations refer to a set of equations that decompose the value function into the immediate reward plus the discounted future values.

- Fallow deterministic policy:

$$
\begin{aligned}
V^{\pi}(s) &= R(s, \pi(s)) + \gamma \sum_{s^{\prime} \in \mathcal{S}} P(s^{\prime}|s, \pi(s)) V^{\pi} (s^{\prime}) \\
Q^{\pi}(s, a) &= R(s, a) + \gamma \sum_{s^{\prime} \in S} P(s^{\prime}|s, a) V^{\pi} (s^{\prime})
\end{aligned}
$$


- Fallow stochastic policy: 
$$
\begin{aligned}
V^{\pi}(s) &= \sum_{a \in A} \pi(a \vert s) R(s, a) + \gamma \sum_{s^{\prime} \in S}\sum_{a \in A} \pi(a \vert s) P(s^{\prime}|s, a) V^{\pi} (s^{\prime}) \\
Q^{\pi}(s, a) &= R(s, a) + \gamma \sum_{s^{\prime} \in S} P(s^{\prime}|s, a) \sum_{a^{\prime} \in A} \pi(a^{\prime} \vert s^{\prime}) Q^{\pi} (s^{\prime}, a^{\prime})
\end{aligned}
$$

# Monte-Carlo Methods

----

- Monte Carlo method uses a simple idea: It learns from episodes of raw experience without modeling the environmental dynamics and computes the observed mean return as an approximation of the expected return.

## Policy Evaluation

- By using $\gamma$ discount sum of reward, the feature reward, known as **return**, is a total sum of discounted rewards going forward. We can compute the return $G_t$ starting from time $t$.
$$
G_t = r_{t+1} + \gamma r_{t+2} + \dots
$$


- Monte Carlo method computes the empirical return by using **complete** episodes, from step $t$ upto terminate $T$:

$$
<s_t, a_t, r_{t+1}, s_{t+1}, a_{t+1}, r_{t+2}, ..., a_{T-1}, r_{T}>
$$


- Monte Carlo requires all the episodes must eventually terminate. To compute empirical return:

$$
G_{t} = \sum_{k=0}^{T-t-1} \gamma ^{k} r_{t+k+1}
$$


- By using empirical return, one can estimate the value function $V(s) = \mathbb{E}[G_t | S_t = s]$ through incremental calculateion ($N(s)$ in this equation is times of state $s$ visited by agent):
$$
\begin{aligned}
V(s) &= \frac{1}{N(s)} (V(s) \dot (N(s) - 1) + G_{t}) \\
&= V(s) + \frac{1}{N(s)} (G_t - V(s))
\end{aligned}
$$


- Similarly:

$$
Q(s, a) = Q(s, a) + \frac{1}{N(s, a)} (G_t - Q(s, a))
$$


- We may count the visit of state $s$ every time so that there could exist multiple visits of one state in one episode ('every-visit'), or only count it the first time we encounter a state in one episode ('first-visit'). 




## Policy Improvement

- Similar with model based's strategy, we improve the policy greedily with respect to the current value function/Q value function:
$$
\pi (s) = arg max_{a \in A} Q(s, a)
$$




## Policy Iteration

- Policy Iteration = Policy evaluation + Policy Improvement
  - Step 1: Initialize $N(s, a) = 0, G(s, a)=0, Q^{\pi }(s, a)=0, \forall s \in S, \forall a \in A$
  - Step 2: Based on episodes $<s_t, a_t, r_{t+1}, ..., s_{T}>$, for each $(s, a)$ pair in all episodes, updating $N(s, a)$, $ G(s, a)$ and $Q^{\pi}(s, a)$ for every time visits $(s, a)$.
  - Step 3: Improve the policy greedily with respect to the current Q value function.
  - Step 4: Generate a new episodes with the new policy $\pi $. (using algorithms $\epsilon$ - greedy helps us balance between exploitation and exploration)

- Pseudocode
  1. $\mathbf{Input}$: $\epsilon $; max episodes $k$  
  1. Initialize $Q(s, a) = 0, N(s, a) = 0 \forall (s, a)$  
  1. Initialize $\pi_{k}$ randomly  
  1. Loop 
  1. $\quad$ Sample $k_{th}$ episode $<s_{k, t}, a_{k, t}, r_{k, t+1}, ..., s_{k, T}>$ given $\pi_{k}$
  1. $\quad$ $G_{k, t} = r_{k, t} + \gamma r_{k, t+1} + \gamma ^{2} r_{k, t+2} + \cdots + \gamma^{T_{k}} r_{k, T_k}$
  1. $\quad$ For $t = 1, ..., T$ do
  1. $\quad$ $\quad$ For every time visit (s, a) in episode k then
  1. $\quad$ $\quad$ $\quad$ $N(s, a) = N(s, a) +1$
  1. $\quad$ $\quad$ $\quad$ $Q(s_t, a_t) = Q(s_t, a_t) + \frac{1}{N(s, a)}(G_{k, t} - Q(s_t, a_t))$
  1. $\quad$ $\quad$ End for
  1. $\quad$ End for
  1. $k = k + 1$
  1. $\pi_{k} = arg max_{a \in A} Q(s_t, a_t)$ with $\epsilon$- greedy
  1. End Loop

## Limilations

- Reducing variance can require a lot of data
- Episodes must be complete


# Temporal Difference Learning

----

“If one had to identify one idea as central and novel to reinforcement learning, it would undoubtedly be temporal-difference (TD) learning.” --- Sutton and Barto 2017

- Temporal Difference(TD) learning can learn from incomplete episodes, just need $<s, a, r, s^{\prime}, a^{\prime}>$ for SARSA, $<s, a, r, s^{\prime}>$ for Q-learning.


## Bootstrapping

TD learning methods update targets with regard to existing estimates rather than exclusively relying on actual rewards and complete returns as in MC methods. This approach is known as bootstrapping. In general, bootstrapping usually refers to a self-starting process that is supposed to proceed without external input. 


## Policy Evaluation

- Relate to the approximation method in Monte Carlo:

$$
\begin{aligned}
V(s) &= V(s) + \frac{1}{N(s)} (G_t - V(s)) \\
Q(s, a) &= Q(s, a) + \frac{1}{N(s, a)} (G_t - Q(s, a))
\end{aligned}
$$

- TD learning is to update the value function/Q-value function towards an estimated return $r_{t+1} + \gamma V(s_{t+1})$ (named 'TD target'). Just replace the $G_t$ in MC method and update the value function by using learning rate hyperparameter $\alpha$:
$$
\begin{aligned}
V(s_t) &= V(s_t) + \alpha ((r_{t+1} + \gamma V(s_{t+1})) - V(s_t)) \\
Q(s_t, a_t) &= Q(s_t, a_t) + \alpha ((r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})) - Q(s_t, a_t))
\end{aligned}
$$


## Policy Improvement and Iteration

- Given an estimated $𝑸^{\pi} (s,a)$, it is convenient update new policy by:

$$
\pi (s) = arg max_{a \in A} Q(s, a)
$$


- There are many famous classic algorithms in learning optimal policy, i.e. SARSA, Q-learning

### SARSA: On-policy algorithm

- **SARSA** refers to the procedure of updaing Q-value by following a sequence of $<s, a, r, s^{\prime}, a^{\prime}>$

- **Step1:** At time step $t$, starting from state $s_t$ and pick action according to $Q$ values, $a_t=argmax_{a\in{A}} Q(s_t, a)$; $\epsilon$-greedy is commonly applied.
- **Step2:** With action $a_t$, we observe reward $r_{t+1}$ and get into the next state $s_{t+1}$.
- **Step3:** Then pick the next action in the same way as in step1 : $a_{t+1} = argmax_{a \in A} Q(s_{t+1}, a)$.
- **Step4:** Update the action-value function:

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha ((r_{t+1} + \gamma Q(s_{t+1}, a_{t+1})) - Q(s_t, a_t))
$$

- Step5: $t=t+1$ and repeat from step1.


- Pseudocode

  1. Initialize $Q(s, a)$  
  1. Repeat (for each episode):  
  1. $\quad$ Initialize $s$  
  1. $\quad$ Choose action $a$ from $s$ using policy derived from $Q$ with $\epsilon$-greedy  
  1. $\quad$ Repeat (for each step of episode):  
  1. $\qquad$ Take action $a$, observe $r$, $s^{\prime}$  
  1. $\qquad$ Choose $a^{\prime}$ from $s^{\prime}$ using policy derived from $Q$ with $\epsilon$-greedy  
  1. $\qquad$ $Q(s, a)$ &larr; $Q(s, a) + \alpha (r + \gamma Q(s^{\prime}, a^{\prime}) - Q(s, a))$  
  1. $\qquad$ $s$ &larr; $s'$; $a$ &larr; $a'$;  
  1. $\quad$ Until $s$ is terminal.  


### Q-Learning: Off-policy algorithm

- Step1: At time step $t$, starting from state $s_t$ and pick action according to $Q$ values, $a_t=argmax_{a\in{A}} Q(s_t, a)$; $\epsilon$-greedy is commonly applied.
- Step2: With action $a_t$, we observe reward $r_{t+1}$ and get into the next state $s_{t+1}$.
- Step3: Update the action-value function:

$$
Q(s_t, a_t) = Q(s_t, a_t) + \alpha ((r_{t+1} + \gamma max_{a \in A}Q(s_{t+1}, a_{t+1})) - Q(s_t, a_t))
$$

- Step4: $t=t+1$ and repeat from step1.


- Pseudocode

  1. Initialize $Q(s, a)$  
  1. Repeat (for each episode):  
  1. $\quad$ Initialize state $s$
  1. $\quad$ Repeat (for each step of episode):  
  1. $\qquad$ Choose $a$ from $s$ using policy derived from $Q$ with $\epsilon$-greedy  
  1. $\qquad$ Take action $a$, observe $r$, $s^{\prime}$  
  1. $\qquad$ $a^{\prime}$ = $argmax_{a \in A} Q(s^{\prime}, a)$  
  1. $\qquad$ $Q(s, a)$ &larr; $Q(s, a) + \alpha (r + \gamma Q(s', a')-Q(s, a))$  
  1. $\qquad$ $s$ &larr; $s^{\prime}$  
  1. $\quad$ Until $s$ is terminal.  